In [1]:
import pathlib
import sys

def _add_src_to_path():
    src_path = str(pathlib.Path.cwd().parent.parent)
    if src_path not in sys.path:
        # sys.path.insert(0, str(pathlib.Path.cwd().parent.parent))
        sys.path.append(str(pathlib.Path.cwd().parent.parent))

_add_src_to_path()

In [7]:
import math
import pathlib

import pandas as pd
import plotly.express as px

from src.utils import torch_utils
from src.vision.models import registry
from src.vision.models import train_model

# import importlib
# importlib.reload(train_model)


def chart_model(model_names: list[str]):
    list_of_data = []
    for model_name in model_names:
        print(f"Model: {model_name}")
        stats_file = pathlib.Path("../../_data") / "vision" / f"_stats_{model_name}.json"
        with open(stats_file, "r") as f:
            epochs = train_model.EpochStatsList.model_validate_json(f.read()).root
            list_of_data.append({
                "model": model_name,
                "epoch": range(len(epochs)),
                "accuracy": [epoch.correct_boards / epoch.total_boards for epoch in epochs],
                "log_loss": [math.log(epoch.loss) for epoch in epochs],
            })
        print(f"  Training Time Per Epoch: {sum(epoch.duration for epoch in epochs) / len(epochs):.2f}s")
        print(f"  Board Accuracy: {epochs[-1].correct_boards / epochs[-1].total_boards}")
        print(f"  Size: {torch_utils.get_model_size(registry.get_model(model_name)).size_in_mb:.2f} MB")
    df = pd.concat([pd.DataFrame(data) for data in list_of_data])
    # Accuracy chart.
    fig = px.line(df, x='epoch', y='accuracy', color='model')
    fig.update_layout(
        yaxis=dict(range=[0, 1]), xaxis_title="Epochs", yaxis_title="Accuracy",
    )
    fig.show()
    # Loss chart.
    fig = px.line(df, x='epoch', y='log_loss', color='model')
    fig.show()


chart_model(["cnnv3", "cnnv2", "cnnv1"])
# chart_model(["cnnv3"])

Model: cnnv3
  Training Time Per Epoch: 77.01s
  Board Accuracy: 0.995
  Size: 9.67 MB
Model: cnnv2
  Training Time Per Epoch: 63.80s
  Board Accuracy: 0.990625
  Size: 4.16 MB
Model: cnnv1
  Training Time Per Epoch: 94.39s
  Board Accuracy: 0.995625
  Size: 49.98 MB
